In [38]:
# importing libraries which are required for dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
from bs4 import BeautifulSoup
import urllib.request

In [40]:
# importing dataset
from google.colab import files
uploaded = files.upload()

Saving iphone12reviews.csv to iphone12reviews (1).csv


In [41]:
import io
iphonereviews = pd.read_csv(io.BytesIO(uploaded['iphone12reviews.csv']))

In [42]:
# Head
iphonereviews.head()

,Unnamed: 0,date_info,name,title,content,rating
0,0,Reviewed in India on 24 December 2020,Ayush taliyan,NOW I ONLY HAVE ONE KIDNEY,My girlfriend demanded me this iphone... And n...,1.0 out of 5 stars
1,1,Reviewed in India on 7 November 2020,Tarun barath,Worst battery,Worst battery performance.. Iphone 11 is far b...,3.0 out of 5 stars
2,2,Reviewed in India on 7 November 2020,S.Siva Ram Kris,"Another Fabulous IPhone ( IPhone 12, Blue 64GB)",Another beauty from Apple. I was upgraded from...,5.0 out of 5 stars
3,3,Reviewed in India on 21 November 2020,Amazon Customer,Kidney as a load balancer,"Sold kidney bought this, now not feeling well ...",4.0 out of 5 stars
4,4,Reviewed in India on 13 November 2020,Paras,The Beast,This was my first switch to an ios device afte...,5.0 out of 5 stars


In [43]:
# Coping data
reviews=iphonereviews.content

In [44]:
reviews=pd.DataFrame(reviews)

In [45]:
# Shape 
reviews.shape

(752, 1)

In [46]:
# Text Blob
from textblob import TextBlob

In [47]:
# Sentiment Analysis
def analyze_sentiment(text):
  analysis=TextBlob(text)
  if analysis.polarity >0 :
    return 'Positive'
  elif analysis.polarity==0:
    return 'Neutral'
  else:
    return 'Negative'

In [48]:
reviews['sentiment']=[str(analyze_sentiment(x))
                          for x in reviews['content']]

In [49]:
reviews.head()

,content,sentiment
0,My girlfriend demanded me this iphone... And n...,Negative
1,Worst battery performance.. Iphone 11 is far b...,Negative
2,Another beauty from Apple. I was upgraded from...,Positive
3,"Sold kidney bought this, now not feeling well ...",Positive
4,This was my first switch to an ios device afte...,Positive


In [50]:
reviews.sentiment.value_counts()

Positive    577
Neutral      88
Negative     87
Name: sentiment, dtype: int64

In [51]:
# Word Analysis
from wordcloud import WordCloud,STOPWORDS

In [52]:
review_words=''
stopwords=set(STOPWORDS)
for val in reviews.content:
  val=str(val)
  tokens=val.split()
  for i in range(len(tokens)):
    tokens[i]=tokens[i].lower()
  review_words +=" ".join(tokens)+" "

In [53]:
wordcloud=WordCloud(width=800,height=800,background_color='white',stopwords=stopwords,
                    min_font_size=10).generate(review_words)

In [54]:
plt.figure(figsize=(8,8))
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [55]:
# TERM FREQUENCT INVERSE DOCUMENT FREQUENCY (TFIDF MATRIX)
from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
tfidf=TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0,
                      stop_words='english',max_features=100)

In [57]:
tfidfmatrix=tfidf.fit_transform(reviews.content)

In [58]:
tfidfmatrix.shape

(752, 100)

In [59]:
df1=pd.DataFrame(tfidfmatrix.toarray(),columns=tfidf.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
tfidfdf=pd.concat([df1,reviews.sentiment],axis=1)

In [61]:
tfidfdf.to_csv('tfidf.csv')

In [62]:
y=tfidfdf.sentiment
X=tfidfdf.drop('sentiment',axis=1)

In [63]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder

In [64]:
le=LabelEncoder()

In [65]:
y=le.fit_transform(y)

In [66]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

In [67]:
logistic=LogisticRegression(multi_class='multinomial')

In [68]:
logisticmodel=logistic.fit(X,y)

In [69]:
logisticmodel.score(X,y)

0.8218085106382979